# Useful code on Pandas DataFrame

## Data Analysis on Salaries_By_College_Major Challenge .

In [2]:
import pandas as pd
pd.set_option('display.width', 400)
pd.set_option('display.max_columns', 12)
pd.options.display.width=None
pd.set_option('display.max_columns', None)

data = pd.read_csv("../data/salaries_by_college_major.csv")
data.head(5)

# print(len(data))
#len(data.drop_duplicates(["Undergraduate Major"]) )



,Undergraduate Major,Starting Median Salary,Mid-Career Median Salary,Mid-Career 10th Percentile Salary,Mid-Career 90th Percentile Salary,Group
0,Accounting,46000.0,77100.0,42200.0,152000.0,Business
1,Aerospace Engineering,57700.0,101000.0,64300.0,161000.0,STEM
2,Agriculture,42600.0,71900.0,36300.0,150000.0,Business
3,Anthropology,36800.0,61500.0,33800.0,138000.0,HASS
4,Architecture,41600.0,76800.0,50600.0,136000.0,Business


## Task 1 to examine / understand the data first before doing any analysis

In [4]:
# Steps to understand the data more, some useful api are 
data.shape
# if there any duplicates on a col 
len(data.drop_duplicates(["Undergraduate Major"])) # len is same so no dup
cols_has_any_null = [ col for col in data.columns if data[col].isnull().any() ]
#print(cols_has_any_null)

col_has_all_null = [ col for col in data.columns if data[col].isnull().all()]
#print(col_has_all_null)


# First drop the row/record which all col as null 
valid_data = data.dropna(how="all")
print(valid_data.tail(5))

after_dropping_col_with_any_null = valid_data.dropna( how="any" ,subset=["Undergraduate Major" , "Group"], 
                                                inplace=False)
#print(after_dropping_col_with_any_null)


      Undergraduate Major  Starting Median Salary  Mid-Career Median Salary  \
46             Psychology                 35900.0                   60400.0   
47               Religion                 34100.0                   52000.0   
48              Sociology                 36500.0                   58200.0   
49                Spanish                 34000.0                   53100.0   
50  Source: PayScale Inc.                     NaN                       NaN   

    Mid-Career 10th Percentile Salary  Mid-Career 90th Percentile Salary Group  
46                            31600.0                           127000.0  HASS  
47                            29700.0                            96400.0  HASS  
48                            30700.0                           118000.0  HASS  
49                            31000.0                            96400.0  HASS  
50                                NaN                                NaN   NaN  


## 1. Find College Major with Highest Starting Salaries

In [5]:
from tabulate import tabulate

max_start_sal = valid_data["Starting Median Salary"].max()
print(f"max_start_sal {max_start_sal}")
college_highest_start_sal = valid_data[ valid_data["Starting Median Salary"] == max_start_sal ]
#print(college_highest_start_sal)
cols = valid_data.columns
print(tabulate(college_highest_start_sal,headers="keys" , tablefmt="pretty"))
#print(tabulate(valid_data))
print(type(college_highest_start_sal))
type(college_highest_start_sal.index)
college_highest_start_sal.loc[college_highest_start_sal.index]


max_start_sal 74300.0
+----+---------------------+------------------------+--------------------------+-----------------------------------+-----------------------------------+-------+
|    | Undergraduate Major | Starting Median Salary | Mid-Career Median Salary | Mid-Career 10th Percentile Salary | Mid-Career 90th Percentile Salary | Group |
+----+---------------------+------------------------+--------------------------+-----------------------------------+-----------------------------------+-------+
| 43 | Physician Assistant |        74300.0         |         91700.0          |              66400.0              |             124000.0              | STEM  |
+----+---------------------+------------------------+--------------------------+-----------------------------------+-----------------------------------+-------+
<class 'pandas.core.frame.DataFrame'>


,Undergraduate Major,Starting Median Salary,Mid-Career Median Salary,Mid-Career 10th Percentile Salary,Mid-Career 90th Percentile Salary,Group
43,Physician Assistant,74300.0,91700.0,66400.0,124000.0,STEM


In [6]:
## some random test on selecting cols on DF

select_1 = valid_data.loc[ valid_data["Group"] == "Business" , ["Undergraduate Major" , "Mid-Career Median Salary" , "Group"] ]
print(select_1)

valid_data["Group"].unique()

valid_data.groupby("Group", dropna=False).count()

# grouping on multiple col , compisite key 
# valid_data.groupby(by=["Group" , "Undergraduate Major"] , dropna=False).count()

sum_result = valid_data.groupby(by=["Group"]).sum()
sum_result.loc[:, ["Starting Median Salary" , "Mid-Career Median Salary"] ]

           Undergraduate Major  Mid-Career Median Salary     Group
0                   Accounting                   77100.0  Business
2                  Agriculture                   71900.0  Business
4                 Architecture                   76800.0  Business
7          Business Management                   72100.0  Business
14                Construction                   88900.0  Business
17                   Economics                   98600.0  Business
22                     Finance                   88300.0  Business
23                    Forestry                   62600.0  Business
27  Health Care Administration                   60600.0  Business
29       Hospitality & Tourism                   57500.0  Business
36                   Marketing                   79600.0  Business
40                     Nursing                   67000.0  Business


,Starting Median Salary,Mid-Career Median Salary
Group,,
Business,535600.0,901000.0
HASS,818100.0,1385300.0
STEM,861800.0,1453000.0


## 2. What college major has the highest mid-career salary? How much do graduates with this major earn?

In [7]:
max_start_sal = valid_data["Mid-Career Median Salary"].max()
print(f"max_start_sal {max_start_sal}")
college_highest_start_sal = valid_data[ valid_data["Mid-Career Median Salary"] == max_start_sal ]
print(college_highest_start_sal)

max_start_sal 107000.0
    Undergraduate Major  Starting Median Salary  Mid-Career Median Salary  \
8  Chemical Engineering                 63200.0                  107000.0   

   Mid-Career 10th Percentile Salary  Mid-Career 90th Percentile Salary Group  
8                            71900.0                           194000.0  STEM  


## 3. Lowest Risk Majors
### A low-risk major is a degree where there is a small difference between the lowest and highest salaries. In other words, if the difference between the 10th percentile and the 90th percentile earnings of your major is small, then you can be more certain about your salary after you graduate.

In [8]:
valid_data["diff_between_percentile"] = valid_data["Mid-Career 90th Percentile Salary"] - valid_data["Mid-Career 10th Percentile Salary"]

print(valid_data.head(5))

print("******")
valid_data.sort_values(by="diff_between_percentile" , ascending=True, inplace=True)
print(valid_data.head(1))

     Undergraduate Major  Starting Median Salary  Mid-Career Median Salary  \
0             Accounting                 46000.0                   77100.0   
1  Aerospace Engineering                 57700.0                  101000.0   
2            Agriculture                 42600.0                   71900.0   
3           Anthropology                 36800.0                   61500.0   
4           Architecture                 41600.0                   76800.0   

   Mid-Career 10th Percentile Salary  Mid-Career 90th Percentile Salary  \
0                            42200.0                           152000.0   
1                            64300.0                           161000.0   
2                            36300.0                           150000.0   
3                            33800.0                           138000.0   
4                            50600.0                           136000.0   

      Group  diff_between_percentile  
0  Business                 109800.0  
1 

## 4. find the degrees with the highest potential? Find the top 5 degrees with the highest values in the 90th percentile. 

In [9]:
valid_data.sort_values(by="Mid-Career 90th Percentile Salary" , ascending=False , inplace=True)
print(valid_data.head(5))

     Undergraduate Major  Starting Median Salary  Mid-Career Median Salary  \
17             Economics                 50100.0                   98600.0   
22               Finance                 47900.0                   88300.0   
8   Chemical Engineering                 63200.0                  107000.0   
37                  Math                 45400.0                   92400.0   
44               Physics                 50300.0                   97300.0   

    Mid-Career 10th Percentile Salary  Mid-Career 90th Percentile Salary  \
17                            50600.0                           210000.0   
22                            47200.0                           195000.0   
8                             71900.0                           194000.0   
37                            45200.0                           183000.0   
44                            56000.0                           178000.0   

       Group  diff_between_percentile  
17  Business                 15940

## Grouping & Pivoting
which category of degrees has the highest average salary? Is it STEM, Business or HASS (Humanities, Arts, and Social Science)?

In [10]:
#valid_data.drop(columns="diff_between_percentile" , inplace=True)
group_wise_avg_sal = valid_data.groupby(by="Group").sum() / valid_data.groupby(by="Group").count()

print(group_wise_avg_sal)

#valid_data.groupby(by="Group").mean()

          Mid-Career 10th Percentile Salary  \
Group                                         
Business                       43566.666667   
HASS                           34145.454545   
STEM                           56025.000000   

          Mid-Career 90th Percentile Salary  Mid-Career Median Salary  \
Group                                                                   
Business                      147525.000000              75083.333333   
HASS                          129363.636364              62968.181818   
STEM                          157625.000000              90812.500000   

          Starting Median Salary  Undergraduate Major  diff_between_percentile  
Group                                                                           
Business            44633.333333                  NaN            103958.333333  
HASS                37186.363636                  NaN             95218.181818  
STEM                53862.500000                  NaN            101600.00

In [ ]:
## Samples of creating DF manually - Data Preparation

In [ ]:
# 1. 
cars = {'Brand': ['Honda Civic','Toyota Corolla','Ford Focus','Audi A4'],
        'Price': [22000,25000,27000,35000],
        'Year': [2015,2013,2018,2018]
        }
 
df = pd.DataFrame(cars)
df